<a href="https://colab.research.google.com/github/giraldeli14/Projeto-ETL-DIO/blob/main/projetoDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto ETL em Python
Nome: Giovana Giraldeli   
Descrição: Projeto final de ETL utilizando Pandas


#Contexto do Projeto

Você trabalha como analista de dados em uma clínica de saúde que atende pacientes em diferentes tipos de tratamento (fisioterapia, pilates, acupuntura, etc.).

A clínica deseja melhorar o acompanhamento dos pacientes e identificar padrões de atendimento, como frequência de sessões, tratamentos mais realizados e pacientes com maior número de faltas.

Atualmente, os dados estão desorganizados em um arquivo CSV simples, e não há nenhum processamento automatizado dessas informações.

Seu papel é criar um pipeline ETL em Python para organizar, tratar e gerar informações úteis a partir desses dados.

#Problema

A clínica possui um arquivo CSV chamado pacientes_tratamentos.csv com os seguintes dados brutos:

* Dados incompletos (valores nulos)
* Tipos de tratamento escritos de formas diferentes
* Datas em formato inconsistente
* Não existe nenhum resumo ou indicador que ajude na tomada de decisão

Exemplo de dados:



```
id_paciente,nome,tratamento,data_sessao,presenca
1,Ana Silva,Fisio,01/03/2025,sim
2,Carlos Souza,fisioterapia,2025-03-01,não
3,Mariana Lima,Acupuntura,02/03/2025,sim
4,Ana Silva,Pilates,05/03/2025,sim
```
Sem um processo ETL, esses dados não são confiáveis e não geram informações úteis para a gestão da clínica.


#Extract

In [9]:
import pandas as pd

def extract_data(pacientes_tratamentos):
    df = pd.read_csv("pacientes_tratamentos.csv")
    return df

# Executar Extract
df_extraido = extract_data("data/pacientes_tratamentos.csv")

df_extraido.head()

,id_paciente,nome,tratamento,data_sessao,presenca
0,1,Ana Silva,Fisio,01/03/2025,sim
1,2,Carlos Souza,Fisioterapia,2025-03-01,não
2,3,Mariana Lima,Acupuntura,02/03/2025,sim
3,4,Ana Silva,Pilates,05/03/2025,sim


#Transform

In [10]:
import pandas as pd

def transform_data(df):
    """
    Realiza a limpeza e transformação dos dados
    """

    # Padronizar nomes dos tratamentos
    df["tratamento"] = df["tratamento"].str.lower()

    df["tratamento"] = df["tratamento"].replace({
        "fisio": "fisioterapia",
        "fisioterapia": "fisioterapia",
        "acupuntura": "acupuntura"
    })

    df["tratamento"] = df["tratamento"].str.capitalize()

    # Converter coluna de presença para booleano
    df["presenca"] = df["presenca"].str.lower().map({
        "sim": True,
        "não": False,
        "nao": False
    })

    # Converter datas para formato padrão
    df["data_sessao"] = pd.to_datetime(df["data_sessao"], errors="coerce")

    # Remover registros com dados inválidos
    df = df.dropna()

    return df

# Executar Transform
df_transformado = transform_data(df_extraido)

df_transformado

,id_paciente,nome,tratamento,data_sessao,presenca
0,1,Ana Silva,Fisioterapia,2025-01-03,True
2,3,Mariana Lima,Acupuntura,2025-02-03,True
3,4,Ana Silva,Pilates,2025-05-03,True


#Load

In [12]:
def load_data(df, caminho_saida):
    df.to_csv(caminho_saida, index=False)

# Executar Load
load_data(df_transformado, "dados_tratados.csv")

print("Dados salvos com sucesso!")




Dados salvos com sucesso!


#Resultado Final

In [13]:
relatorio = (
    df_transformado
    .groupby("tratamento")
    .agg(
        total_sessoes=("presenca", "count"),
        presencas=("presenca", "sum")
    )
)

relatorio



,total_sessoes,presencas
tratamento,,
Acupuntura,1,1
Fisioterapia,1,1
Pilates,1,1
